### земля - планета / земля - вещество
Любым способом выполните следующее задание (можно писать свой код, можно запускать готовый код, модифицировать код, который вы писали на курсе по машинному обучению, результат подавать на вход другому готовому коду или сравнивать с электронными лексикографическими ресурсами). 
Опишите всю цепочку вычислений. Задание. 

Соберите корпус, на котором можно обучаться для различения двух значений лексемы (лексему см. в табличке https://goo.gl/8YU1Dr) 

(NB: можно воспользоваться любым поисковиком, выдачей НКРЯ по специальным запросам - например, для датасета со словом ключ (см. датасет с семинарского занятия) я воспользовалась контекстами (-15--7 /+10 +15) "вода|бить|лес..." и т.п.; метод, которым вы создадите тренировочный корпус оценивается). 

Важно, чтобы метод создания датасета позволял рсставить значения автоматически, а не вручную.
Датасет должен состоять примерно из 300 предложений. Составьте свой собственный тестовый сет из 8 предложений. 

Любыми двумя способами соберите признаки, на основе которых можно было бы различить два значения (PMI, Word2Vec, WordNet ...). 
Обучите классифкатор распознавать значения. 

Результат: топ 10 хороших признаков для различения значения 1 и значения 2; 
описание процедур, которые Вы применили или собирались применить для нахождения этих признаков (например, воспользовались word2vec - только укажите подробности, как именно); 
точность, полнота, F мера для кросс-валидации. 
Результат работы классификатора на Ваших 8 предложениях

### создание корпуса
- выбрала слова для коллокаций из ruSKELL
    - для "планеты": планета|космос|орбита|космический|астероид|вращаться|марс|венера|спутник
    - для "вещества": засыпать|присыпать|пахать|рыть|копать|песок|вода|сырой|дерновый|влажный|пложородный|горшок|закапывать|зарывать|выкапывать
- поиск по контекстам (-10, +10) в НКРЯ

взято: 150 с первым значением и 150 со вторым

### программа

In [23]:
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics

In [2]:
m = Mystem()

### загрузка стопслов

In [3]:
def load_stop_words(stop_filename):
    with open(stop_filename, encoding = 'utf-8') as f:
        stopwords = [w.strip() for w in f.readlines() if w.strip()]
    return set(stopwords)

In [4]:
stop_words = load_stop_words("stoplist_russian.txt")
print(stop_words)

{'после', 'даже', 'главный', 'рамка', 'по', 'едва', 'а', 'этот', 'ли', 'самый', 'восемь', 'не', 'лишь', 'что', 'уже', 'над', 'в', 'он', 'со', 'на', 'примечательно', 'первый', 'пока', 'ещё', 'два', 'по-прежнему', 'я', 'еще', 'она', 'каждый', 'с', 'без', 'и', 'здесь', 'стать', 'во', 'год', 'четыре', 'свой'}


### загрузка данных, препроцессинг

In [5]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

#### обучающие данные

In [6]:
df_train = pd.read_csv('earth_soil.csv', encoding='utf-8', sep = '\t')
df_train = df_train[['text', 'sense']]
df_train.head()

,text,sense
0,Благодаря запуску на орбиту вокруг Земли специ...,планета
1,Земля (орбита Земли) есть мера всех орбит.,планета
2,"Как видно из данного рисунка, между магнитными...",планета
3,Сопоставление ВГП Солнца на поверхности планет...,планета
4,"Широтные параметры смещений, видимо, надо иска...",планета


In [7]:
df_train.shape

(300, 2)

In [8]:
for i in range(len(df_train)):
    processed_text = lemmatize(df_train.iloc[i]["text"])
    processed_text_2 = []
    for word in processed_text:
        if (len(word) > 0 
            and word not in stop_words 
            and word != "земля"):
            processed_text_2.append(word)
    df_train.iloc[i]["text"] = " ".join(processed_text_2)
df_train.head()

,text,sense
0,благодаря запуск орбита вокруг специализирован...,планета
1,орбита быть мера весь орбита,планета
2,как видно из данный рисунок между магнитный по...,планета
3,сопоставление вгп солнце поверхность планета и...,планета
4,широтный параметр смещение видимо надо искать ...,планета


#### тестовые данные

In [9]:
df_test = pd.read_csv('earth_soil_test.csv', encoding='utf-8', sep = '\t')
df_test = df_test[['text', 'sense']]
df_test.head()

,text,sense
0,"В год столетия Сергея Павловича Королева, 150-...",планета
1,"При этом оказывается, что центроид оболочки си...",планета
2,"Например, для Меркурия n = 10, для Венеры n =1...",планета
3,Спутник плывет над сияющей Землей; колониальны...,планета
4,"Годится почвенная смесь, просеянная через мелк...",вещество


In [10]:
df_test.shape

(8, 2)

In [11]:
for i in range(len(df_test)):
    processed_text = lemmatize(df_test.iloc[i]["text"])
    processed_text_2 = []
    for word in processed_text:
        if (len(word) > 0 
            and word not in stop_words 
            and word != "земля"):
            processed_text_2.append(word)
    df_test.iloc[i]["text"] = " ".join(processed_text_2)
df_test.head()

,text,sense
0,столетие сергей павлович королева 150 летие ко...,планета
1,при это оказываться центроид оболочка система ...,планета
2,например для меркурий n 10 для венера n 11 для...,планета
3,спутник плыть сиять колониальный народ бунтова...,планета
4,годиться почвенный смесь просеивать через мелк...,вещество


### векторизация (tf-idf)

In [24]:
#vec = TfidfVectorizer(analyzer='char', ngram_range=(1,8), norm=None, use_idf=False, binary=True, max_df = 600)
vec = TfidfVectorizer(analyzer='word', ngram_range=(1,3), use_idf=True, binary=True, norm='l2')
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.sense)
y_train = label_enc.transform(df_train.sense)

feature matrix shape (300, 11961)


### обучение модели (баесовский классификатор)

In [25]:
# fit our prediction model
model = MultinomialNB(alpha=1.0)
model.fit(X, y_train)

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.sense)
X_pred = model.predict(X_test)

## кросс-валидация

In [38]:
from sklearn.model_selection import cross_val_score

print ('precision', np.mean(cross_val_score(model, X, y_train, cv=10, scoring='precision')))
print ('recall', np.mean(cross_val_score(model, X, y_train, cv=10, scoring='recall')))
print ('f1', np.mean(cross_val_score(model, X, y_train, cv=10, scoring='f1')))

precision 0.969485294118
recall 0.986666666667
f1 0.977176028921


## результат на тестовых данных

In [39]:
print('accuracy', metrics.accuracy_score(y_test, X_pred))

accuracy 1.0


In [40]:
print("настоящие значения", y_test)
print("предсказания модели", X_pred)

настоящие значения [1 1 1 1 0 0 0 0]
предсказания модели [1 1 1 1 0 0 0 0]
